In [3]:
import pandas as pd
import numpy as np
import xgboost as xgb

In [11]:
data2014 = pd.read_csv("finalized/2014-scores.csv")
data2016 = pd.read_csv("finalized/2016-scores.csv")
data2018 = pd.read_csv("finalized/2018-scores.csv")